# Pre-Work

## Mount Google Drive and Change Directory

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd gdrive/MyDrive/Machine_Learning/Code/

/content/gdrive/MyDrive/Machine_Learning/Code


## Install Modules


In [ ]:
!pip install torch

In [ ]:
!pip install pyvi

     |████████████████████████████████| 8.5 MB 4.4 MB/s 
     |████████████████████████████████| 743 kB 44.5 MB/s 


In [ ]:
!pip install jsonl_to_conll

In [ ]:
! pip install datasets transformers seqeval

     |████████████████████████████████| 270 kB 5.3 MB/s 
     |████████████████████████████████| 2.8 MB 33.2 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 1.3 MB 43.7 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 243 kB 42.0 MB/s 
     |████████████████████████████████| 119 kB 42.9 MB/s 
     |████████████████████████████████| 3.3 MB 31.7 MB/s 
     |████████████████████████████████| 636 kB 38.2 MB/s 
     |████████████████████████████████| 895 kB 53.5 MB/s 
     |████████████████████████████████| 294 kB 46.1 MB/s 
     |████████████████████████████████| 142 kB 42.3 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=a39d734a729c61a08ca56a3b5e6aa10630e361280d52885254af10e99d54ea8c
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall:

## Pip Show Modules

In [ ]:
pip show transformers

Name: transformers
Version: 4.10.2
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Sam Shleifer, Patrick von Platen, Sylvain Gugger, Suraj Patil, Stas Bekman, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: /usr/local/lib/python3.7/dist-packages
Requires: packaging, sacremoses, importlib-metadata, numpy, requests, tqdm, tokenizers, filelock, regex, pyyaml, huggingface-hub
Required-by: 


In [ ]:
pip show seqeval

Name: seqeval
Version: 1.2.2
Summary: Testing framework for sequence labeling
Home-page: https://github.com/chakki-works/seqeval
Author: Hironsan
Author-email: hiroki.nakayama.py@gmail.com
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: scikit-learn, numpy
Required-by: 


## Import Module

In [ ]:
import transformers
print(transformers.__version__)

4.10.2


In [ ]:
#@title Importing Machine Learning Library
import torch
import transformers
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import tokenization_utils_base as tub
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from sklearn.model_selection import train_test_split

In [ ]:
#@title Importing tqdm
from tqdm import tqdm, trange

In [ ]:
#@title Importing Vietnamese Tokenizer
from pyvi import ViTokenizer

In [ ]:
#@title import json_to_conll
from jsonl_to_conll import convert

In [ ]:
#@title import pytorch dataset
from datasets import load_dataset, load_metric
from datasets import Dataset
import datasets

In [ ]:
#@title Importing common Python libraries
import pandas as pd
import io
import ast
import os
import json
import numpy as np
#from pyvi import ViTokenizer
import matplotlib.pyplot as plt
from collections import OrderedDict

# Preprocess the data

## Read labeled data 1 as dataframe

We read the first labeled data

In [ ]:
with open('/content/gdrive/My Drive/Machine_Learning/Code/Data/BERT_CTEL/Labeled_CTEL_Data1.jsonl') as f:
    lines = f.read().splitlines()

In [ ]:
df_CTEL = pd.DataFrame(lines)
df_CTEL.columns = ['json_element']

In [ ]:
df_CTEL['json_element'].apply(json.loads)

0       {'id': 2, 'data': 'Hà Luxury ; 3 tháng 1 , 201...
1       {'id': 3, 'data': 'BIDV Thành Đô', 'label': [[...
2       {'id': 4, 'data': 'Nikken Sekkei ; Architect &...
3       {'id': 5, 'data': 'Obayashi Viet Nam ; Archite...
4       {'id': 6, 'data': 'Obayashi Design Partner ; A...
                              ...                        
1158    {'id': 1698, 'data': 'BAO MINH ; 9 tháng 5 , 2...
1159    {'id': 1699, 'data': 'Geniee International Pte...
1160    {'id': 1700, 'data': 'TPBank ; 2 tháng 10 , 20...
1161    {'id': 1701, 'data': 'FIS ; 13 tháng 10 , 2014...
1162    {'id': 12218, 'data': 'Lend Lease Tyco Joint V...
Name: json_element, Length: 1163, dtype: object

In [ ]:
df_CTEL_final1 = pd.json_normalize(df_CTEL['json_element'].apply(json.loads))

In [ ]:
for i in range(len(df_CTEL_final1['label'].tolist())):
  for num in range(len(df_CTEL_final1['label'][i])):
    if df_CTEL_final1['label'][i][num][2] == 'company':
      df_CTEL_final1['label'][i][num][2] = df_CTEL_final1['label'][i][num][2].replace('company', 'COM')
    if df_CTEL_final1['label'][i][num][2] == 'COMPANY':
      df_CTEL_final1['label'][i][num][2] = df_CTEL_final1['label'][i][num][2].replace('COMPANY', 'COM')
    if df_CTEL_final1['label'][i][num][2] == 'job':
      df_CTEL_final1['label'][i][num][2] = df_CTEL_final1['label'][i][num][2].replace('job', 'JOB')

In [ ]:
df_CTEL_final1.head(5)

,id,data,label
0,2,"Hà Luxury ; 3 tháng 1 , 2019 - Hiện tại","[[0, 9, COM]]"
1,3,BIDV Thành Đô,"[[0, 15, COM]]"
2,4,Nikken Sekkei ; Architect & Urban Design - Kiế...,"[[16, 25, JOB], [28, 40, JOB], [43, 52, JOB], ..."
3,5,Obayashi Viet Nam ; Architect ; Thành phố Hồ C...,"[[0, 17, COM], [20, 29, JOB]]"
4,6,Obayashi Design Partner ; Architect ; Shinagaw...,"[[0, 23, COM], [26, 35, JOB]]"


In [ ]:
df_CTEL_final1.shape

(1163, 3)

## Read labeled data 2 as dataframe

We read the second labeled data

In [ ]:
with open('/content/gdrive/My Drive/Machine_Learning/Code/Data/BERT_CTEL/Labeled_CTEL_Data2.jsonl') as f:
    lines = f.read().splitlines()

In [ ]:
df_CTEL2 = pd.DataFrame(lines)
df_CTEL2.columns = ['json_element']

In [ ]:
df_CTEL2['json_element'].apply(json.loads)

0       {'id': 56815, 'data': 'Ho chi minh city ; Appl...
1       {'id': 56816, 'data': 'Dimark Australia ; Dire...
2       {'id': 56817, 'data': 'Dimark Australia Pty Lt...
3       {'id': 56819, 'data': 'University of Transport...
4       {'id': 56820, 'data': 'Trần Minh Thương Hiệu M...
                              ...                        
1399    {'id': 58857, 'data': 'PayPal ; Full-Stack Eng...
1400    {'id': 58858, 'data': 'Berkeley City College ;...
1401    {'id': 58859, 'data': 'UC Berkeley ; Lab Assis...
1402    {'id': 58860, 'data': 'APU International Schoo...
1403    {'id': 58862, 'data': 'Minh Son HealthCare Co ...
Name: json_element, Length: 1404, dtype: object

In [ ]:
df_CTEL_final2 = pd.json_normalize(df_CTEL2['json_element'].apply(json.loads))
df_CTEL_final2.head(5)

,id,data,label
0,56815,Ho chi minh city ; Applications Developer ; Th...,"[[19, 41, JOB]]"
1,56816,"Dimark Australia ; Director ; 10 tháng 10 , 20...","[[0, 16, COM], [19, 27, JOB]]"
2,56817,Dimark Australia Pty Ltd ; Director ; Sydney,"[[0, 24, COM], [27, 35, JOB]]"
3,56819,University of Transport and Communications ; D...,"[[45, 60, JOB]]"
4,56820,Trần Minh Thương Hiệu Môi Giới BĐS Số 1 ; Nhà ...,"[[42, 54, JOB]]"


In [ ]:
df_CTEL_final2.shape

(1404, 3)

## Read labeled CTEL 2000 only-company data as dataframe

We read the 2000 only-company data

In [ ]:
df_CTEL_comp = pd.read_csv('/content/gdrive/My Drive/Machine_Learning/Code/Data/BERT_CTEL/Labeled_CTEL_2000_OnlyCompany.csv')

In [ ]:
df_CTEL_comp.head(15)

,Unnamed: 0,Sentence #,Word,Tag
0,0,Sentence: 1,CÔNG,B-COM
1,1,Sentence: 1,TY,I-COM
2,2,Sentence: 1,TNHH,I-COM
3,3,Sentence: 1,TỔNG,I-COM
4,4,Sentence: 1,HỢP,I-COM
5,5,Sentence: 1,N,I-COM
6,6,Sentence: 1,K,I-COM
7,7,Sentence: 1,T,I-COM
8,8,Sentence: 2,B&K,B-COM
9,9,Sentence: 2,VINA,I-COM


In [ ]:
df_CTEL_comp.shape

(33372, 4)

## Preporcess 2000 only-company dataframe

In [ ]:
clone_CTEL_comp = df_CTEL_comp
token_list = clone_CTEL_comp.groupby('Sentence #')['Word'].apply(pd.Series.tolist).tolist()
nertag_list = clone_CTEL_comp.groupby('Sentence #')['Tag'].apply(pd.Series.tolist).tolist()

In [ ]:
nertag_list[0]

['B-COM', 'I-COM', 'I-COM', 'I-COM', 'I-COM', 'I-COM', 'I-COM', 'I-COM']

In [ ]:
token_list[0]

['CÔNG', 'TY', 'TNHH', 'TỔNG', 'HỢP', 'N', 'K', 'T']

In [ ]:
' '.join(map(str, token_list[0]))

'CÔNG TY TNHH TỔNG HỢP N K T'

In [ ]:
company_list = [' '.join(map(str, i)) for i in token_list]

In [ ]:
len(company_list)

1998

## Merge the first and second labeled dataframe


In [ ]:
df_CTEL_final = pd.concat([df_CTEL_final1, df_CTEL_final2])

In [ ]:
df_CTEL_final.iloc[2200]

id                                                   58316
data     Hanoi - Amsterdam Art Team [ HAT ] ; Vice Pres...
label                                      [[37, 51, JOB]]
Name: 1037, dtype: object

In [ ]:
df_CTEL_final.shape

(2567, 3)

In [ ]:
type(df_CTEL_final['label'])

pandas.core.series.Series

## Using json_to_conll to generate ner_tags for the first and second labeled dataframe

In [ ]:
clone_CTEL = df_CTEL_final.drop(columns = ['id']).rename(columns={'data': 'text', 'label': 'labels'})
clone_CTEL['text'] = clone_CTEL['text'].str.lower()
clone_CTEL.head()

,text,labels
0,"hà luxury ; 3 tháng 1 , 2019 - hiện tại","[[0, 9, COM]]"
1,bidv thành đô,"[[0, 15, COM]]"
2,nikken sekkei ; architect & urban design - kiế...,"[[16, 25, JOB], [28, 40, JOB], [43, 52, JOB], ..."
3,obayashi viet nam ; architect ; thành phố hồ c...,"[[0, 17, COM], [20, 29, JOB]]"
4,obayashi design partner ; architect ; shinagaw...,"[[0, 23, COM], [26, 35, JOB]]"


In [ ]:
clone_CTEL.shape

(2567, 2)

In [ ]:
label_list = ['O', 'B-COM', 'I-COM', 'B-JOB', 'I-JOB']

In [ ]:
#@title Convert ner_tags into int
dict_label_list = {label_list[i]:i for i in range(len(label_list))}
dict_num_to_label = {i:label_list[i] for i in range(len(label_list))}

In [ ]:
#@title Change tokens in each sentence into corresponding ner_tags
ner_tags = []
tokens = []
for i in range(clone_CTEL.shape[0]):
  arr = convert.flatten(dict(clone_CTEL.iloc[i]))
  ner_tags.append([dict_label_list[tup[1]] for tup in arr])
  tokens.append([tup[0] for tup in arr])
d = {'ner_tags': ner_tags, 'tokens': tokens}
bert_CTEL_df = pd.DataFrame(d)
bert_CTEL_df.head()

,ner_tags,tokens
0,"[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[hà, luxury, ;, 3, tháng, 1, ,, 2019, -, hiện,..."
1,"[1, 2, 2]","[bidv, thành, đô]"
2,"[1, 2, 0, 3, 0, 3, 4, 0, 3, 4, 0, 3, 4, 4, 4, ...","[nikken, sekkei, ;, architect, &, urban, desig..."
3,"[1, 2, 2, 0, 3, 0, 0, 0, 0, 0, 0]","[obayashi, viet, nam, ;, architect, ;, thành, ..."
4,"[1, 2, 2, 0, 3, 0, 0, 0, 0, 0, 0]","[obayashi, design, partner, ;, architect, ;, s..."


## Generate ner_tags for 2000 only-company dataframe

In [ ]:
train_comp_dataframe = pd.DataFrame({'ner_tags': [[dict_label_list[i] for i in j] for j in nertag_list], 'tokens': token_list})

In [ ]:
train_comp_dataframe.head(10)

,ner_tags,tokens
0,"[1, 2, 2, 2, 2, 2, 2, 2]","[CÔNG, TY, TNHH, TỔNG, HỢP, N, K, T]"
1,"[1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[T.B.K, CO.,, LTD, ,, CÔNG, TY, TNHH, THƯƠNG, ..."
2,"[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[CÔNG, TY, TNHH, DỊCH, VỤ, THƯƠNG, MẠI, DƯỢC, ..."
3,"[1, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, ...","[VAN, GIAO, SOLAR, POWER, PLANT, COMPANY, LIMI..."
4,"[1, 2, 0, 1, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, ...","[VIETNAMFOOD, CO.,LTD, ,, VIETNAMFOOD, CO.,LTD..."
5,"[1, 2, 2, 2, 0, 1, 2, 2, 2, 0, 1, 2, 2, 2, 2, ...","[CAO, GIA, MAN, CO.,LTD, ,, CAO, GIA, MAN, CO...."
6,"[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[CÔNG, TY, TRÁCH, NHIỆM, HỮU, HẠN, MỘT, THÀNH,..."
7,"[1, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 0, 1, ...","[T, DUNG, CO.,, LTD, ,, Tran, Dung, Quality, A..."
8,"[1, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, ...","[ARC, GENTLE, CONSTRUCTION, CONSULTING, SERVIC..."
9,"[1, 2, 2, 2, 0, 1, 2, 2, 2, 0, 1, 2, 2, 2, 2, ...","[TRAN, GIA, PHAT, IMEXTRADICO, ,, TRAN, GIA, P..."


# Convert data to fit the model's input

## Generate PyTorch Train, Validation Dataset

Here, we would want only full sentence (included job title) on the valid and test dataset. Therefore, we would treat all of the only-company data as training data

In [ ]:
bert_CTEL_df.shape

(2567, 2)

In [ ]:
train_df, test_df = train_test_split(bert_CTEL_df, test_size=0.2)
valid_df, test_df = train_test_split(test_df, test_size=0.5)

## Export test dataframe

In [ ]:
test_df.to_csv('/content/gdrive/My Drive/Machine_Learning/Code/Data/Dataset_CTEL/Test_CTEL_company.csv')

FileNotFoundError: ignored

## Export train dataframe

In [ ]:
train_df.to_csv('/content/gdrive/My Drive/Machine_Learning/Code/Data/Dataset_CTEL/train_CTEL_company.csv')

## Export valid dataframe

In [ ]:
valid_df.to_csv('/content/gdrive/My Drive/Machine_Learning/Code/Data/Dataset_CTEL/valid_CTEL_company.csv')

## Merge train dataframe with 2000 only-company dataframe

In [ ]:
train_df.shape

(2053, 2)

In [ ]:
clone_train_df = pd.concat([train_df, train_comp_dataframe])

In [ ]:
dataset_train = Dataset.from_pandas(train_df)
dataset_valid = Dataset.from_pandas(valid_df)
#dataset_test = Dataset.from_pandas(test_df)

In [ ]:
dataset_train

Dataset({
    features: ['ner_tags', 'tokens', '__index_level_0__'],
    num_rows: 2053
})

In [ ]:
dataset_train_comp = Dataset.from_pandas(clone_train_df)
dataset_train_comp

Dataset({
    features: ['ner_tags', 'tokens', '__index_level_0__'],
    num_rows: 4051
})

In [ ]:
#dataset_train_comp['tokens']

In [ ]:
t_arr = dataset_train_comp['tokens']
for i in range(len(t_arr)):
  n = False
  for j in range(len(t_arr[i])):
    if t_arr[i][j] == 'None' or t_arr[i][j] == 'NONE' or t_arr[i][j] == None:
      n = True
  if n == True:
    dataset_train_comp['tokens'][i] = dataset_train_comp['tokens'][i].remove(None)
    print(i, " ", dataset_train_comp['tokens'][i], " ")
    print(dataset_train_comp['tokens'][i][:-1])

2432   ['BQ', 'VINA', 'CO.,', 'LTD', ',', 'BQ', 'VINA', 'CO.,', 'LTD', ',', 'CÔNG', 'TY', 'TNHH', 'BẢO', 'QUÂN', 'VI', None]  
['BQ', 'VINA', 'CO.,', 'LTD', ',', 'BQ', 'VINA', 'CO.,', 'LTD', ',', 'CÔNG', 'TY', 'TNHH', 'BẢO', 'QUÂN', 'VI']
2576   [None, 'RU', 'VINA', 'CO.,', 'LTD', ',', 'CÔNG', 'TY', 'TNHH', None, 'RU', 'VI', None]  
[None, 'RU', 'VINA', 'CO.,', 'LTD', ',', 'CÔNG', 'TY', 'TNHH', None, 'RU', 'VI']


In [ ]:
dataset_train_comp = dataset_train_comp.filter(lambda ex: ex['tokens'].count(None) == 0)

  0%|          | 0/5 [00:00<?, ?ba/s]

## Import Train Dataframe

In [ ]:
CTEL_train_df = pd.read_csv('/content/gdrive/My Drive/Machine_Learning/Code/Data/Dataset_CTEL/train_CTEL_company.csv')

In [ ]:
CTEL_train_df['ner_tags'] = CTEL_train_df['ner_tags'].apply(lambda x: json.loads(x))
CTEL_train_df['tokens'] = CTEL_train_df['tokens'].apply(lambda x: ast.literal_eval(x))

In [ ]:
CTEL_train_df = CTEL_train_df.drop(columns = ['Unnamed: 0'])
CTEL_train_df.head()

,ner_tags,tokens
0,"[1, 2, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[arasaka, sarl, ;, team, leader, ;, tháng, 5, ..."
1,"[0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0]","[health, management, associates, ;, principal,..."
2,"[1, 2, 2, 2, 2, 2]","[công, ty, truyền, thông, toàn, dũng]"
3,"[0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[hanoi, medical, university, ;, lecturer, ;, t..."
4,"[0, 0, 0, 3, 4]","[hà, nội, ;, sinh, viên]"


## Preprocess 4000 more only-company data

Here, besided 2000 companies that we have for the training, we could add 4000 companies for the training to see if we can achieve the better result

### Import 4000 only-company DataFrame

In [ ]:
CTEL_cutcomp_df = pd.read_csv('/content/gdrive/My Drive/Machine_Learning/Code/Data/BERT_CTEL/Labeled_CTEL_4000_OnlyCompany.csv')

In [ ]:
CTEL_cutcomp_df.head()

,brief_name,eng_name,name
0,-1,S&t Logistics And Trading Service Company Limited,Công Ty TNHH Tiếp Vận Và Dịch Vụ Thương Mại S&t
1,-1,-1,CÔNG TY TRÁCH NHIỆM HỮU HẠN THƯƠNG MẠI VẬN TẢI...
2,-1,-1,CÔNG TY TNHH BÁCH THI
3,CTY TNHH MTV DƯƠNG ĐÔNG - GIA LAI,CTY TNHH MTV DƯƠNG ĐÔNG - GIA LAI,CÔNG TY TNHH MỘT THÀNH VIÊN DƯƠNG ĐÔNG - GIA LAI
4,-1,-1,CÔNG TY TNHH THƯƠNG MẠI DỊCH VỤ VẬT LIỆU XÂY D...


In [ ]:
CTEL_cutcomp_df.shape

(4000, 3)

### Assign ner-tag for 4000 only-company

In [ ]:
sr_bn = CTEL_cutcomp_df['brief_name'][CTEL_cutcomp_df['brief_name'] != '-1']

In [ ]:
sr_en = CTEL_cutcomp_df['eng_name'][CTEL_cutcomp_df['eng_name'] != '-1']

In [ ]:
sr_n = CTEL_cutcomp_df['name'][CTEL_cutcomp_df['name'] != '-1']

In [ ]:
cutcomp_dict = {}
cutcomp_dict['ner_tags'] = []
cutcomp_dict['tokens'] = []

In [ ]:
#tokenizer_cutcomp = AutoTokenizer.from_pretrained('vinai/phobert-base')

In [ ]:
sr_bn_l = sr_bn.tolist()
for i in range(len(sr_bn_l)):
  tok_sen = ViTokenizer.tokenize(sr_bn_l[i])
  tok_arr = tok_sen.split(' ')
  ner = []
  for j in range(len(tok_arr)):
    if j == 0:
      ner.append(1)
    else:
      ner.append(2)
  cutcomp_dict['ner_tags'].append(ner)
  cutcomp_dict['tokens'].append(tok_arr)

In [ ]:
sr_en_l = sr_en.tolist()
for i in range(len(sr_en_l)):
  tok_sen = ViTokenizer.tokenize(sr_en_l[i])
  tok_arr = tok_sen.split(' ')
  ner = []
  for j in range(len(tok_arr)):
    if j == 0:
      ner.append(1)
    else:
      ner.append(2)
  cutcomp_dict['ner_tags'].append(ner)
  cutcomp_dict['tokens'].append(tok_arr)

In [ ]:
sr_n_l = sr_n.tolist()
for i in range(len(sr_n_l)):
  tok_sen = ViTokenizer.tokenize(sr_n_l[i])
  tok_arr = tok_sen.split(' ')
  ner = []
  for j in range(len(tok_arr)):
    if j == 0:
      ner.append(1)
    else:
      ner.append(2)
  cutcomp_dict['ner_tags'].append(ner)
  cutcomp_dict['tokens'].append(tok_arr)

In [ ]:
CTEL_labeled_cutcomp_df = pd.DataFrame(cutcomp_dict)
CTEL_labeled_cutcomp_df.shape

(8482, 2)

In [ ]:
# CTEL_labeled_cutcomp_df['ner_tags'] = CTEL_labeled_cutcomp_df['ner_tags'].apply(lambda x: json.loads(x))
# CTEL_labeled_cutcomp_df['tokens'] = CTEL_labeled_cutcomp_df['tokens'].apply(lambda x: ast.literal_eval(x))

In [ ]:
CTEL_labeled_cutcomp_df.head()

,ner_tags,tokens
0,"[1, 2, 2, 2, 2, 2, 2]","[CTY_TNHH, MTV, DƯƠNG, ĐÔNG, -, GIA, LAI]"
1,"[1, 2, 2, 2, 2]","[KIEN, TRUC, ADH, ,, JSC]"
2,"[1, 2, 2, 2]","[MTV, ADS., ,, JSC]"
3,"[1, 2, 2, 2, 2, 2]","[CTY_TNHH, TT, &, DV, VIETSHOW, CÀ_MAU]"
4,"[1, 2, 2, 2, 2]","[G., D., S, COMPANY, LIMITED]"


### Merge the train dataframe with the labeled 4000 only-company dataframe

In [ ]:
CTEL_train_df = pd.concat([clone_train_df, CTEL_labeled_cutcomp_df])
CTEL_train_df.shape

(12533, 2)

In [ ]:
# CTEL_train_df['ner_tags'] = CTEL_train_df['ner_tags'].apply(lambda x: json.loads(x))
# CTEL_train_df['tokens'] = CTEL_train_df['tokens'].apply(lambda x: ast.literal_eval(x))

In [ ]:
CTEL_train_df.head()

,ner_tags,tokens
981,"[0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0, 0, 0, 0, ...","[clb, học, làm, giàu, ., hce, ;, phó, chủ, nhi..."
1474,"[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[fpt, telecom, ;, support, services, ;, 27, th..."
1551,"[0, 0, 0, 3, 4, 4, 4, 0, 3, 4, 0, 0, 0, 0, 0, ...","[hà, nội, ;, tư, vấn, thuế, , &, kế, toán, ;, ..."
1712,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, ...","[giải, pháp, phần, mềm, trọn, gói, game, net, ..."
2450,"[1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[bigtrees, ;, ceo, ;, 14, tháng, 10, ,, 2019, ..."


In [ ]:
CTEL_train_df.shape

(12533, 2)

### Count company and job tags

In [ ]:
c_comp = 0
c_job = 0
for i in CTEL_train_df['ner_tags'].tolist():
  for arr in i:
    if dict_num_to_label[arr] == 'B-COM' or dict_num_to_label[arr] == 'I-COM':
      c_comp = c_comp + 1
    if dict_num_to_label[arr] == 'B-JOB' or dict_num_to_label[arr] == 'I-JOB':
      c_job = c_job + 1
print(c_comp , " ", c_job)

82473   3802


## Import Validation Dataframe

In [ ]:
CTEL_valid_df = pd.read_csv('/content/gdrive/My Drive/Machine_Learning/Code/Data/Dataset_CTEL/valid_CTEL_company.csv')

FileNotFoundError: ignored

In [ ]:
CTEL_valid_df['ner_tags'] = CTEL_valid_df['ner_tags'].apply(lambda x: json.loads(x))
CTEL_valid_df['tokens'] = CTEL_valid_df['tokens'].apply(lambda x: ast.literal_eval(x))

In [ ]:
CTEL_valid_df = CTEL_valid_df.drop(columns = ['Unnamed: 0'])
CTEL_valid_df.head()

,ner_tags,tokens
0,"[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[vtc, intecom, ;, tháng, 8, năm, 2009, -, thán..."
1,"[0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, ...","[autonomous, intelligent, robotics, lab, ;, te..."
2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[google, ;, 2020, -, hiện, tại, ;, austin, ,, ..."
3,"[1, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[pi-top, ;, software, engineer, ;, tháng, 9, n..."
4,"[0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[les, éclaireurs, ;, chroniqueuse, ;, tháng, 9..."


## Import Test Dataframe

In [ ]:
CTEL_test_df = pd.read_csv('/content/gdrive/My Drive/Machine_Learning/Code/Data/Dataset_CTEL/Test_CTEL_company.csv')

In [ ]:
CTEL_test_df['ner_tags'] = CTEL_test_df['ner_tags'].apply(lambda x: json.loads(x))
CTEL_test_df['tokens'] = CTEL_test_df['tokens'].apply(lambda x: ast.literal_eval(x))

In [ ]:
CTEL_test_df = CTEL_test_df.drop(columns= ['Unnamed: 0'])

In [ ]:
CTEL_test_df.head()

,ner_tags,tokens
0,"[1, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[google, ;, software, engineering, intern, ;, ..."
1,"[0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0]","[rue, des, chats, ;, co-founder, &, designer, ..."
2,"[1, 0, 3, 4, 0, 0]","[inescid, ;, postdoctoral, researcher, ;, lisboa]"
3,"[1, 2, 2, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, ...","[seta, international, llc, ;, director, of, vn..."
4,"[0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[vietseeds, foundation, ;, sinh, viên, ;, 19, ..."


## Remove None in the tokens array

In [ ]:
def remove_none(string):
  index = 0
  while index < len(string):
    if not isinstance(string[index], str):
      string.pop(index)
    else:
      index = index + 1
  return string

In [ ]:
test = dataset_train['tokens'][2576]
test

['RU', 'VINA', 'CO.,', 'LTD', ',', 'CÔNG', 'TY', 'TNHH', None, 'RU', 'VI']

In [ ]:
CTEL_train_df['tokens'] = CTEL_train_df['tokens'].map(remove_none)

## Create Dataset from Processed Dataframes

In [ ]:
dataset_train = Dataset.from_pandas(CTEL_train_df)

In [ ]:
dataset_train = dataset_train.remove_columns('__index_level_0__')
dataset_train

Dataset({
    features: ['ner_tags', 'tokens'],
    num_rows: 12533
})

In [ ]:
#dataset_valid = Dataset.from_pandas(CTEL_valid_df)
dataset_valid = Dataset.from_pandas(valid_df)
dataset_valid = dataset_valid.remove_columns('__index_level_0__')

In [ ]:
dataset_valid

Dataset({
    features: ['ner_tags', 'tokens'],
    num_rows: 257
})

In [ ]:
#dataset_test = Dataset.from_pandas(CTEL_test_df)
dataset_test = Dataset.from_pandas(test_df)
dataset_test = dataset_test.remove_columns('__index_level_0__')

In [ ]:
dataset_test

Dataset({
    features: ['ner_tags', 'tokens'],
    num_rows: 257
})

## Create Dataset Dict

In [ ]:
temp_dataset = {'train': dataset_train, 'validation': dataset_valid, 'test': dataset_test} 
dataset_dict = datasets.DatasetDict(temp_dataset)
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['ner_tags', 'tokens'],
        num_rows: 12533
    })
    validation: Dataset({
        features: ['ner_tags', 'tokens'],
        num_rows: 257
    })
    test: Dataset({
        features: ['ner_tags', 'tokens'],
        num_rows: 257
    })
})

## Define model tokenizer

In [ ]:
model_checkpoint = 'bert-base-multilingual-uncased'
batch_size = 16

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

## Tokenize dataset

In [ ]:
label_all_tokens = True

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
test_function = tokenize_and_align_labels(dataset_train)

In [ ]:
tokenized_datasets = dataset_dict.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'ner_tags', 'token_type_ids', 'tokens'],
        num_rows: 12533
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'ner_tags', 'token_type_ids', 'tokens'],
        num_rows: 257
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'ner_tags', 'token_type_ids', 'tokens'],
        num_rows: 257
    })
})

# Fine-tuning BERT model


## Initialize the model

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Downloading:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint a

In [ ]:
#@title Initialize arguments
args = TrainingArguments(
    "test-ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
)

In [ ]:
#@title Specify tokenizer for the model
data_collator = DataCollatorForTokenClassification(tokenizer)

#Define the metric for the model

In [ ]:
metric = load_metric("seqeval")

Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## Initialize the Tranier parameters

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

## Train model

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags.
***** Running training *****
  Num examples = 12533
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7840


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.098000,0.155774,0.807971,0.846300,0.826691,0.951622
2,0.027300,0.175983,0.782895,0.903226,0.838767,0.952198


Saving model checkpoint to test-ner/checkpoint-500
Configuration saved in test-ner/checkpoint-500/config.json
Model weights saved in test-ner/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags.
***** Running Evaluation *****
  Num examples = 257
  Batch size = 16
Saving model checkpoint to test-ner/checkpoint-1000
Configuration saved in test-ner/checkpoint-1000/config.json
Model weights saved in test-ner/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to test-ner/checkpoint-1500
Configuration saved in test-ner/checkpoint-1500/confi

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.098000,0.155774,0.807971,0.846300,0.826691,0.951622
2,0.027300,0.175983,0.782895,0.903226,0.838767,0.952198
3,0.017200,0.155943,0.849722,0.869070,0.859287,0.961989
4,0.008900,0.217303,0.836007,0.889943,0.862132,0.957957
5,0.006900,0.223369,0.833977,0.819734,0.826794,0.954502
6,0.004300,0.254812,0.841210,0.844402,0.842803,0.956038
7,0.002900,0.284776,0.814159,0.872865,0.842491,0.953926
8,0.001700,0.253180,0.824561,0.891841,0.856882,0.956422
9,0.001200,0.268093,0.850662,0.853890,0.852273,0.958725
10,0.000900,0.273118,0.840074,0.867173,0.853408,0.958725


Saving model checkpoint to test-ner/checkpoint-2000
Configuration saved in test-ner/checkpoint-2000/config.json
Model weights saved in test-ner/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-2000/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-2000/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags.
***** Running Evaluation *****
  Num examples = 257
  Batch size = 16
Saving model checkpoint to test-ner/checkpoint-2500
Configuration saved in test-ner/checkpoint-2500/config.json
Model weights saved in test-ner/checkpoint-2500/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-2500/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-2500/special_tokens_map.json
Saving model checkpoint to test-ner/checkpoint-3000
Configuration saved in test-ner/checkpoint-3000/

TrainOutput(global_step=7840, training_loss=0.015022798234178703, metrics={'train_runtime': 2765.5137, 'train_samples_per_second': 45.319, 'train_steps_per_second': 2.835, 'total_flos': 2391015401100690.0, 'train_loss': 0.015022798234178703, 'epoch': 10.0})

## Evaluating the model

In [ ]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags.
***** Running Evaluation *****
  Num examples = 257
  Batch size = 16


{'epoch': 10.0,
 'eval_accuracy': 0.958725283163755,
 'eval_f1': 0.8534080298786181,
 'eval_loss': 0.27311787009239197,
 'eval_precision': 0.8400735294117647,
 'eval_recall': 0.8671726755218216,
 'eval_runtime': 2.3372,
 'eval_samples_per_second': 109.958,
 'eval_steps_per_second': 7.274}

## Test model 

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets['test'])
tp = predictions
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, ner_tags.
***** Running Prediction *****
  Num examples = 257
  Batch size = 16


{'COM': {'f1': 0.7969924812030075,
  'number': 201,
  'precision': 0.803030303030303,
  'recall': 0.7910447761194029},
 'JOB': {'f1': 0.9307958477508651,
  'number': 289,
  'precision': 0.9307958477508651,
  'recall': 0.9307958477508651},
 'overall_accuracy': 0.9664096916299559,
 'overall_f1': 0.8761514841351074,
 'overall_precision': 0.8788501026694046,
 'overall_recall': 0.8734693877551021}

In [ ]:
tokenized_datasets['test']

Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'ner_tags', 'token_type_ids', 'tokens'],
    num_rows: 257
})

In [ ]:
#tokenized_datasets['test'][0]

In [ ]:
predictions[0]

array([0, 1, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0])

## Test Model on two sentences

In [ ]:
input_list = ['Viki ; Software engineer ; Tháng 7 năm 2012 - Tháng 9 năm 2015 ; I help build data warehousing infrastructure to power data-driven decision makings .',
              'Facebook App ; Software Engineer Intern ; 12 tháng 9 , 2011 - Tháng 12 năm 2011 ; Palo Alto , California ; I help make sure more people get addicted to Facebook .']
input_sent_tokenized = tokenizer(input_list)
input_sent_dataset = Dataset.from_dict(input_sent_tokenized)
input_sent_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2
})

In [ ]:
trainer.predict(input_sent_dataset)

***** Running Prediction *****
  Num examples = 2
  Batch size = 16


PredictionOutput(predictions=array([[[ 8.315423  , -1.5289447 , -1.4321797 , -2.2866697 ,
         -2.1449525 ],
        [-0.8006873 ,  8.733067  , -2.7793155 , -2.1335983 ,
         -2.3822906 ],
        [-0.9074506 ,  8.7835865 , -2.6680508 , -2.080744  ,
         -2.46403   ],
        [ 9.215715  , -1.7647337 , -1.6866707 , -2.2454174 ,
         -2.3687897 ],
        [-2.7342637 , -1.7406482 , -3.2353528 ,  7.4153214 ,
         -0.4248885 ],
        [-2.244118  , -3.2882094 , -2.1482177 , -1.4110627 ,
          7.5055137 ],
        [ 9.343124  , -1.7776976 , -1.7596852 , -2.3995585 ,
         -2.1534934 ],
        [ 9.309216  , -1.6950047 , -1.8141186 , -2.3205185 ,
         -2.405958  ],
        [ 9.28714   , -1.7099355 , -1.7738662 , -2.3636518 ,
         -2.3444846 ],
        [ 9.289382  , -1.7684693 , -1.7884524 , -2.50986   ,
         -2.2596307 ],
        [ 9.269591  , -1.7846547 , -1.6793778 , -2.4144545 ,
         -2.2872372 ],
        [ 9.30026   , -1.7384697 , -1.829365  ,

## Save Model

In [ ]:
trainer.save_model('/content/gdrive/My Drive/Machine_Learning/Code/CTEL_BERT_Official_CompJob')

# Perform model on test data

## Read data as dataframe


In [ ]:
CTEL_df = pd.read_excel('/content/gdrive/My Drive/Machine_Learning/Code/Data/Dataset_CTEL/dataset_ctel1.xlsx')
CTEL_df.head()

,Sốđiệnthoại,Họ tên,Giới tính,Địa chỉ,link fb,Ghi chú chung,Sản phẩm khảo sát,Ghi chú 1,Nhật ký cuộc gọi,Ghi chú 2,Khách hàng đang sử dụng hệ thống email từ nhà cung cấp nào?,Ghi chú 3,Số lượng nhân viên,Ghi chú 4,Ghi chú khảo sát,Thời gian gọi gần nhất,Người gọi,Link ghi âm,Trạng thái
0,901665009,Buy Home,Công ty TNHH Rubyphone\nManager\nThành phố Hồ ...,Tỉnh/Thành phố hiện tại\nThành phố Hồ Chí Minh...,https://www.facebook.com/profile.php?id=100010...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Trống
1,903611595,Minh Trí Võ,Thành phố Hồ Chí Minh\ngiám đốc\n2000 - Hiện tại,Tỉnh/Thành phố hiện tại\nThành phố Hồ Chí Minh,https://www.facebook.com/profile.php?id=100003...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Trống
2,935256333,Dương Lê (Phụ Kiện Tín Thành),Phụ Kiện Điện Thoại TÍN THÀNH\nGiám đốc\nĐông Hà,Tỉnh/Thành phố hiện tại\nĐông Hà\n\nQuê quán\n...,https://www.facebook.com/profile.php?id=100005...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-02 03:38:06.073,Mai Thị Thùy,https://media.api-connect.io:443/record/5d9397...,Không có nhu cầu
3,948901169,Ngô Vũ Thành Lợi,"Khoduan.net\nFounder\n1 tháng 3, 2019 - Hiện t...",Tỉnh/Thành phố hiện tại\nHà Nội\n\nQuê quán\nH...,https://www.facebook.com/profile.php?id=100005...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Trống
4,399555633,Đăng Khánh (Khanh Gà),Stanley Electric\nDirector (company)\n2007 - H...,Tỉnh/Thành phố hiện tại\nHà Nội\n\nQuê quán\nG...,https://www.facebook.com/profile.php?id=100003...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Trống


In [ ]:
company = CTEL_df['Giới tính'].str.replace('\n', '; ').tolist()

In [ ]:
company = [[i] for i in company]

In [ ]:
company[0][0]

'Công ty TNHH Rubyphone; Manager; Thành phố Hồ Chí Minh; Phân Phối Sỉ Linh Phụ Kiện Apple, Đồ Chơi Công Nghệ Số.....'

In [ ]:
tokenizer('Công ty TNHH Rubyphone; Manager; Thành phố Hồ Chí Minh; Phân Phối Sỉ Linh Phụ Kiện Apple, Đồ Chơi Công Nghệ Số.....')

{'input_ids': [101, 11638, 17053, 42906, 10243, 10243, 36340, 34735, 132, 14964, 132, 11333, 14061, 10616, 11581, 13718, 132, 12097, 44939, 10193, 20976, 14016, 18272, 17006, 117, 11042, 19176, 11638, 19246, 10297, 119, 119, 119, 119, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
d_comp = {}
d_comp['input_ids'] = []
d_comp['token_type_ids'] = []
d_comp['attention_mask']  = []
d_comp['tokens'] = []
d_comp

{'attention_mask': [], 'input_ids': [], 'token_type_ids': [], 'tokens': []}

In [ ]:
for i in company:
  temp = tokenizer(i[0])
  d_comp['input_ids'].append(temp['input_ids'])
  d_comp['token_type_ids'].append(temp['token_type_ids'])
  d_comp['attention_mask'].append(temp['attention_mask'])
  d_comp['tokens'].append(tokenizer.convert_ids_to_tokens(temp['input_ids']))

In [ ]:
len(d_comp['tokens'])

6497

In [ ]:
len(d_comp['input_ids'])

6497

In [ ]:
print(d_comp['input_ids'][0])
print(d_comp['token_type_ids'][0])
print(d_comp['attention_mask'][0])
print(d_comp['tokens'][0])

[101, 11638, 17053, 42906, 10243, 10243, 36340, 34735, 132, 14964, 132, 11333, 14061, 10616, 11581, 13718, 132, 12097, 44939, 10193, 20976, 14016, 18272, 17006, 117, 11042, 19176, 11638, 19246, 10297, 119, 119, 119, 119, 119, 102]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
['[CLS]', 'cong', 'ty', 'tn', '##h', '##h', 'ruby', '##phone', ';', 'manager', ';', 'thanh', 'pho', 'ho', 'chi', 'minh', ';', 'phan', 'phoi', 'si', 'linh', 'phu', 'kien', 'apple', ',', 'đo', 'choi', 'cong', 'nghe', 'so', '.', '.', '.', '.', '.', '[SEP]']


In [ ]:
Dataset_comp = Dataset.from_dict(d_comp)
Dataset_comp

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'tokens'],
    num_rows: 6497
})

## Test Model

In [ ]:
def transform_dataset(d):
  t_arr_in_ids = []
  t_arr_in_ids.append(d['input_ids'])
  d['input_ids'] = t_arr_in_ids
  t_arr_tok_ids = []
  t_arr_tok_ids.append(d['token_type_ids'])
  d['token_type_ids'] = t_arr_tok_ids
  t_arr_atten_ids = []
  t_arr_atten_ids.append(d['attention_mask'])
  d['attention_mask'] = t_arr_atten_ids
  t_arr_token = []
  t_arr_token.append(d['tokens'])
  d['tokens'] = t_arr_token
  return Dataset.from_dict(d)

In [ ]:
pred, _, _ = trainer.predict(Dataset_comp)

In [ ]:
pred = np.argmax(pred, axis=2)

In [ ]:
pred.shape

(6497, 285)

In [ ]:
pred[0]

array([0, 1, 2, 2, 2, 2, 2, 2, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
#Dataset_comp[0]['tokens']

In [ ]:
result = []
for i in range(len(Dataset_comp)):
  label_in_tag = [dict_num_to_label[i] for i in pred[i]]
  token_label = [(t,l) for (t,l) in zip(Dataset_comp[i]['tokens'], label_in_tag)]
  result.append(token_label)

In [ ]:
add_dict = {}
add_dict['company'] = []
add_dict['job_title'] = []

In [ ]:
for array in result:
  comp = ""
  job = ""
  for i in array:
    if i[1] == 'B-COM' or i[1] == 'I-COM':
      if i[0][0] == "#":
        comp = comp + i[0]
      else:
        comp = comp + " " + i[0]
    if i[1] == 'B-JOB' or i[1] == 'I-JOB':
      if i[0][0] == "#":
        job = job + i[0]
      else:
        job = job + " " + i[0]
  comp = comp.replace("#", "")
  job = job.replace("#", "")
  add_dict['company'].append(comp)
  add_dict['job_title'].append(job)

In [ ]:
len(add_dict['company'])

6497

In [ ]:
len(add_dict['job_title'])

6497

## Merge Feature

In [ ]:
CTEL_df['company'] = add_dict['company']
CTEL_df['job_title'] = add_dict['job_title']

In [ ]:
CTEL_df.head(15)

,Sốđiệnthoại,Họ tên,Giới tính,Địa chỉ,link fb,Ghi chú chung,Sản phẩm khảo sát,Ghi chú 1,Nhật ký cuộc gọi,Ghi chú 2,Khách hàng đang sử dụng hệ thống email từ nhà cung cấp nào?,Ghi chú 3,Số lượng nhân viên,Ghi chú 4,Ghi chú khảo sát,Thời gian gọi gần nhất,Người gọi,Link ghi âm,Trạng thái,company,job_title
0,901665009,Buy Home,Công ty TNHH Rubyphone\nManager\nThành phố Hồ ...,Tỉnh/Thành phố hiện tại\nThành phố Hồ Chí Minh...,https://www.facebook.com/profile.php?id=100010...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Trống,cong ty tnhh rubyphone,manager
1,903611595,Minh Trí Võ,Thành phố Hồ Chí Minh\ngiám đốc\n2000 - Hiện tại,Tỉnh/Thành phố hiện tại\nThành phố Hồ Chí Minh,https://www.facebook.com/profile.php?id=100003...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Trống,,giam đoc
2,935256333,Dương Lê (Phụ Kiện Tín Thành),Phụ Kiện Điện Thoại TÍN THÀNH\nGiám đốc\nĐông Hà,Tỉnh/Thành phố hiện tại\nĐông Hà\n\nQuê quán\n...,https://www.facebook.com/profile.php?id=100005...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-02 03:38:06.073,Mai Thị Thùy,https://media.api-connect.io:443/record/5d9397...,Không có nhu cầu,,giam đoc
3,948901169,Ngô Vũ Thành Lợi,"Khoduan.net\nFounder\n1 tháng 3, 2019 - Hiện t...",Tỉnh/Thành phố hiện tại\nHà Nội\n\nQuê quán\nH...,https://www.facebook.com/profile.php?id=100005...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Trống,,founder founder sales manager sales manager
4,399555633,Đăng Khánh (Khanh Gà),Stanley Electric\nDirector (company)\n2007 - H...,Tỉnh/Thành phố hiện tại\nHà Nội\n\nQuê quán\nG...,https://www.facebook.com/profile.php?id=100003...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Trống,stanley electric,director assistant
5,379410099,Ng Đình Hoà,Hà Nội\nTổng Giám đốc\n2011 - Hiện tại,Tỉnh/Thành phố hiện tại\nHà Nội\n\nQuê quán\nH...,https://www.facebook.com/profile.php?id=100015...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Trống,,tong giam đoc
6,327678222,Võ Xuân Đức (Gà Đỏ),"Cty winpro unza vietnam\n2 tháng 5, 2019 - Hiệ...",Tỉnh/Thành phố hiện tại\nĐắk Mil\n\nQuê quán\n...,https://www.facebook.com/profile.php?id=100004...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Trống,cty winpro unza vietnam,tribal chief
7,987532012,Như Quỳnh Triệu (Ord Taobao),Triệu Như Quỳnh Order\nFounder/Manager\n31 thá...,Tỉnh/Thành phố hiện tại\nHải Phòng,https://www.facebook.com/profile.php?id=100015...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Trống,,founder manager
8,348147969,Hồ Ngọc Hân,Pờ Kinh Hội\ngiám đốc công ty me bô'ng con\nBa...,NaN,https://www.facebook.com/profile.php?id=100005...,dập máy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-01 09:16:20.850,Ngô Diệu Linh,https://media.api-connect.io:443/record/5d9397...,Sai số,cong ty me bo ' ng con,giam đoc
9,965963173,Aley Võ (Thẩm Mỹ Toàn Diện),"Aley Cosmetic Clinic\nManager\n26 tháng 2, 202...",Tỉnh/Thành phố hiện tại\nThành phố Hồ Chí Minh...,https://www.facebook.com/profile.php?id=100013...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Trống,,manager


# Export Dataframe as Excel

In [ ]:
os.getcwd()

'/content/gdrive/My Drive/Machine_Learning/Code/Data/Dataset_CTEL'

In [ ]:
#os.chdir('./gdrive/My Drive/Machine_Learning/Code/Data/Dataset_CTEL/')

In [ ]:
writer = pd.ExcelWriter('Duong_CTEL1_VER2_Labeled.xlsx')

In [ ]:
CTEL_df.to_excel(writer)

In [ ]:
writer.save()